In [ ]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.15.1 requires cffi>=1.16.0, but you have cffi 1.15.1 which is incompatible.


In [ ]:
from mip import *
from queue import Queue

In [ ]:
def read(filename):
    """
    Lê um arquivo de texto contendo dados de um problema de programação linear inteira e
    retorna o número de variáveis, o número de restrições, o vetor objetivo e as restrições.

    Parâmetros:
    filename (string): O nome do arquivo a ser lido.

    Retorna:
    tuple: Contendo o número de variáveis, o número de restrições, o vetor da função objetivo e as restrições.
    """

    # Abre o arquivo para leitura
    with open(filename, 'r') as file:
        # Lê todas as linhas do arquivo e remove espaços extras
        lines = file.readlines()
        lines = [line.strip().split() for line in lines]  # Quebra cada linha em uma lista de elementos
        # Converte todos os elementos das listas de strings para inteiros
        lines = [[int(element) for element in line] for line in lines]

        # A primeira linha contém o número de variáveis e o número de restrições
        variables = lines[0][0]
        num_restrictions = lines[0][1]

        # A segunda linha contém o vetor da função objetivo
        objetive = lines[1]

        # As demais linhas representam as restrições do problema
        restrictions = lines[2:]

    return variables, num_restrictions, objetive, restrictions

In [ ]:
def solve_relaxation(model, restricoes, objetivo, variaveis):
    """
    Resolve a relaxação linear do problema configurando o modelo com a função
    objetivo e restrições fornecidadas.

    Parâmetros:
    model (mip.Model): O modelo de programação linear inteira a ser otimizado.
    restricoes (list): Lista de listas representando as restrições, onde cada
                       sublista contém os coeficientes das variáveis e o valor
                       do lado direito.
    objetivo (list): Lista de coeficientes para a função objetivo.
    variaveis (int): O número total de variáveis no problema.

    Retorna:
    tuple: Retorna o valor da função objetivo e os valores fracionários das variáveis
           se houver uma solução; caso contrário, retorna (None, None).
    """

    # Define a função objetivo do modelo como a soma dos produtos dos coeficientes
    # do objetivo e das variáveis correspondentes
    model.objective = xsum(objetivo[i] * model.vars[i] for i in range(variaveis))

    # Adiciona cada restrição ao modelo, onde cada uma consiste na soma dos produtos
    # dos coeficientes das variáveis, limitados pelo valor do lado direito da restrição
    for data in restricoes:
        model += xsum(data[i] * model.vars[i] for i in range(variaveis)) <= data[-1]

    # Executa o processo de otimização para encontrar a melhor solução fracionária
    model.optimize()

    # Se o modelo encontrar uma solução, retorna o valor da função objetivo e a
    # lista dos valores fracionários das variáveis; caso contrário, retorna (None, None)
    if model.num_solutions:
        return model.objective_value, [v.x for v in model.vars]
    else:
        return None, None

In [ ]:
def branch(solucao, alpha=0.0001):
    """
    Identifica a variável mais próxima de ser binária, mas que ainda não é inteira, para ser usada
    como a variável de ramificação no processo de Branch and Bound.

    Parâmetros:
    solucao (list): Lista de valores das variáveis na solução atual.
    alpha (float, opcional): Tolerância para considerar uma variável como inteira. O valor padrão é 0.0001.

    Retorna:
    int ou None: O índice da variável de ramificação se houver uma variável não binária;
                 caso contrário, retorna None.
    """

    # Identifica as variáveis não binárias (fracionárias) com base no critério alpha
    nao_binarias = [(i, abs(x - round(x))) for i, x in enumerate(solucao) if abs(x - round(x)) > alpha]

    # Se todas as variáveis são consideradas binárias dentro da tolerância, retorna None
    if not nao_binarias:
        return None

    # Ordena as variáveis fracionárias por proximidade de 0.5 (mais longe de serem inteiras)
    nao_binarias.sort(key=lambda x: -abs(x[1] - 0.5))

    # Seleciona o índice da variável de ramificação mais distante de 0.5
    branchV = nao_binarias[0][0]
    return branchV


In [ ]:
def branch_and_bound(filename):
    """
    Implementa o algoritmo Branch and Bound para resolver um problema de programação inteira.
    Lê os dados do problema a partir de um arquivo, resolve o modelo relaxado e, se necessário,
    realiza ramificações para buscar uma solução ótima.

    Parâmetros:
    filename (str): O nome do arquivo contendo os dados do problema.

    Retorna:
    tuple: O valor da função objetivo da melhor solução inteira e a melhor solução encontrada.
    """

    # Lê os dados do problema: número de variáveis, número de restrições, função objetivo e restrições
    variaveis, num_restricoes, objetivo, restricoes = read(filename)

    # Inicializa uma fila para armazenar nós do Branch and Bound
    queue = Queue()
    # Coloca o nó raiz na fila, sem restrições adicionais e com integridade inicial para cada variável
    queue.put(([], {i: None for i in range(variaveis)}))

    best_solution = None
    best_objective = float('-inf')

    # Enquanto houver nós na fila para explorar
    while not queue.empty():
        restricao_atual, integridade_atual = queue.get()

        # Cria um novo modelo de otimização com sentido de maximização
        m = Model(sense=MAXIMIZE)
        # Define as variáveis contínuas com limites entre 0 e 1
        m.vars = [m.add_var(var_type=CONTINUOUS, lb=0, ub=1) for i in range(variaveis)]

        # Adiciona as restrições fixas do nó atual ao modelo
        for (x, y) in restricao_atual:
            if y is not None:
                m += m.vars[x] == y

        # Resolve a relaxação linear do modelo atual
        objective_value, solution = solve_relaxation(m, restricoes, objetivo, variaveis)

        # Se o valor objetivo da relaxação linear é pior que o melhor encontrado, ignora o ramo
        if objective_value is None or objective_value <= best_objective:
            continue

        # Se a solução é viável e todos os valores das variáveis são binários, atualiza a melhor solução
        if all(x in (0, 1) for x in solution):
            if objective_value > best_objective:
                best_solution = solution
                best_objective = objective_value
            continue

        # Determina a próxima variável de ramificação
        branch_var = branch(solution)
        if branch_var is not None:
            # Ramifica nas variáveis binárias (0 ou 1), adicionando restrições e colocando novos nós na fila
            for i in [0, 1]:
                nova_restricao = restricao_atual + [(branch_var, i)]
                queue.put((nova_restricao, integridade_atual))

    return best_objective, best_solution


In [ ]:
def print_solution(optimal_value, optimal_solution):
    if optimal_value is not None:
        print(f"Optimal value: {optimal_value}")
        print("Optimal solution:")

        for i, x in enumerate(optimal_solution):
            print(f"x[{i}] = {x}")
    else:
        print("No feasible solution found.")

In [ ]:
print_solution(*branch_and_bound('test1.txt'))

Optimal value: 20.0
Optimal solution:
x[0] = 0.0
x[1] = 0.0
x[2] = 0.0
x[3] = 0.0
x[4] = 1.0
x[5] = 1.0
x[6] = 0.0


In [ ]:
print_solution(*branch_and_bound('test2.txt'))

Optimal value: 24.0
Optimal solution:
x[0] = 0.0
x[1] = 0.0
x[2] = 0.0
x[3] = 0.0
x[4] = 0.0
x[5] = 1.0
x[6] = 1.0
x[7] = 0.0
x[8] = 1.0


In [ ]:
print_solution(*branch_and_bound('test3.txt'))

Optimal value: 19.0
Optimal solution:
x[0] = 0.0
x[1] = 0.0
x[2] = 1.0
x[3] = 0.0
x[4] = 0.0
x[5] = 0.0
x[6] = 1.0
x[7] = 0.0
x[8] = 0.0


In [ ]:
print_solution(*branch_and_bound('test4.txt'))

Optimal value: 10.0
Optimal solution:
x[0] = 0.0
x[1] = 0.0
x[2] = 1.0
x[3] = 0.0
x[4] = 0.0
x[5] = 0.0
x[6] = 0.0
x[7] = 0.0
x[8] = 0.0
